In [66]:
import os
import glob
import psycopg2
import pandas as pd
from sql_queries import *

In [67]:
conn = psycopg2.connect("host=127.0.0.1 dbname=sparkifydb user=student password=student")
cur = conn.cursor()

In [68]:
def get_files(filepath):
    all_files = []
    for root, dirs, files in os.walk(filepath):
        files = glob.glob(os.path.join(root,'*.json'))
        for f in files :
            all_files.append(os.path.abspath(f))
    
    return all_files

# Process `song_data`
In this first part, you'll perform ETL on the first dataset, `song_data`, to create the `songs` and `artists` dimensional tables.

Let's perform ETL on a single song file and load a single record into each table to start.
- Use the `get_files` function provided above to get a list of all song JSON files in `data/song_data`
- Select the first song in this list
- Read the song file and view the data

In [69]:
song_files = "/home/workspace/data/song_data"

In [70]:
filepath = "/home/workspace/data/song_data/A/A/A"
get_files(filepath)[1]

'/home/workspace/data/song_data/A/A/A/TRAAARJ128F9320760.json'

In [71]:
df = pd.read_json(path_or_buf=get_files(filepath)[2], lines = True)
print(df.tail())

            artist_id  artist_latitude              artist_location  \
0  AR10USD1187B99F3F1              NaN  Burlington, Ontario, Canada   

   artist_longitude            artist_name   duration  num_songs  \
0               NaN  Tweeterfriendly Music  189.57016          1   

              song_id         title  year  
0  SOHKNRJ12A6701D1F8  Drop of Rain     0  


In [72]:
df[['artist_id','artist_name','artist_location','artist_latitude','artist_longitude']]

,artist_id,artist_name,artist_location,artist_latitude,artist_longitude
0,AR10USD1187B99F3F1,Tweeterfriendly Music,"Burlington, Ontario, Canada",NaN,NaN


In [73]:
df[['song_id','title','artist_id','year','duration']]

,song_id,title,artist_id,year,duration
0,SOHKNRJ12A6701D1F8,Drop of Rain,AR10USD1187B99F3F1,0,189.57016


In [74]:
df1 = df[['song_id','title','artist_id','year','duration']]
df1.values.tolist()

[['SOHKNRJ12A6701D1F8', 'Drop of Rain', 'AR10USD1187B99F3F1', 0, 189.57016]]

## #1: `songs` Table
#### Extract Data for Songs Table
- Select columns for song ID, title, artist ID, year, and duration
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `song_data`

In [75]:
song_data = df1.values.tolist()[0]
song_data

['SOHKNRJ12A6701D1F8', 'Drop of Rain', 'AR10USD1187B99F3F1', 0, 189.57016]

In [76]:
print(songplay_table_drop)
print(song_table_insert)

DROP TABLE if exists songplays;
Insert into songs  values (%s, %s, %s, %s, %s) on conflict (song_id) do nothing ;


#### Insert Record into Song Table
Implement the `song_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song into the `songs` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songs` table in the sparkify database.

In [77]:
cur.execute(song_table_insert, song_data)
conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

## #2: `artists` Table
#### Extract Data for Artists Table
- Select columns for artist ID, name, location, latitude, and longitude
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `artist_data`

In [78]:
df2 = df[['artist_id','artist_name','artist_location','artist_latitude','artist_longitude']]
artist_data = df2.values.tolist()[0]
artist_data


['AR10USD1187B99F3F1',
 'Tweeterfriendly Music',
 'Burlington, Ontario, Canada',
 nan,
 nan]

#### Insert Record into Artist Table
Implement the `artist_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song's artist into the `artists` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `artists` table in the sparkify database.

In [79]:
cur.execute(artist_table_insert, artist_data)
conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

# Process `log_data`
In this part, you'll perform ETL on the second dataset, `log_data`, to create the `time` and `users` dimensional tables, as well as the `songplays` fact table.

Let's perform ETL on a single log file and load a single record into each table.
- Use the `get_files` function provided above to get a list of all log JSON files in `data/log_data`
- Select the first log file in this list
- Read the log file and view the data

In [80]:
log_files = "/home/workspace/data/log_data/"

In [81]:
filepath = "/home/workspace/data/log_data/2018/11"

In [82]:
df = pd.read_json(path_or_buf=get_files(filepath)[2], lines = True)
df.head()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,Frumpies,Logged In,Anabelle,F,0,Simpson,134.47791,free,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",PUT,NextSong,1.541044e+12,455,Fuck Kitty,200,1541903636796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",69
1,Kenny G with Peabo Bryson,Logged In,Anabelle,F,1,Simpson,264.75057,free,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",PUT,NextSong,1.541044e+12,455,By The Time This Night Is Over,200,1541903770796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",69
2,Biffy Clyro,Logged In,Anabelle,F,2,Simpson,189.83138,free,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",PUT,NextSong,1.541044e+12,455,God & Satan,200,1541904034796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",69
3,None,Logged In,Lily,F,0,Burns,NaN,free,"New York-Newark-Jersey City, NY-NJ-PA",GET,Home,1.540621e+12,456,None,200,1541910841796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",32
4,HIM,Logged In,Lily,F,1,Burns,212.06159,free,"New York-Newark-Jersey City, NY-NJ-PA",PUT,NextSong,1.540621e+12,456,Beautiful,200,1541910973796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",32


## #3: `time` Table
#### Extract Data for Time Table
- Filter records by `NextSong` action
- Convert the `ts` timestamp column to datetime
  - Hint: the current timestamp is in milliseconds
- Extract the timestamp, hour, day, week of year, month, year, and weekday from the `ts` column and set `time_data` to a list containing these values in order
  - Hint: use pandas' [`dt` attribute](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.dt.html) to access easily datetimelike properties.
- Specify labels for these columns and set to `column_labels`
- Create a dataframe, `time_df,` containing the time data for this file by combining `column_labels` and `time_data` into a dictionary and converting this into a dataframe

In [83]:
df = pd.read_json(path_or_buf=get_files(filepath)[2], lines = True)
df

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,Frumpies,Logged In,Anabelle,F,0,Simpson,134.47791,free,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",PUT,NextSong,1.541044e+12,455,Fuck Kitty,200,1541903636796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",69
1,Kenny G with Peabo Bryson,Logged In,Anabelle,F,1,Simpson,264.75057,free,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",PUT,NextSong,1.541044e+12,455,By The Time This Night Is Over,200,1541903770796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",69
2,Biffy Clyro,Logged In,Anabelle,F,2,Simpson,189.83138,free,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",PUT,NextSong,1.541044e+12,455,God & Satan,200,1541904034796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",69
3,None,Logged In,Lily,F,0,Burns,NaN,free,"New York-Newark-Jersey City, NY-NJ-PA",GET,Home,1.540621e+12,456,None,200,1541910841796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",32
4,HIM,Logged In,Lily,F,1,Burns,212.06159,free,"New York-Newark-Jersey City, NY-NJ-PA",PUT,NextSong,1.540621e+12,456,Beautiful,200,1541910973796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",32
5,Matmos,Logged In,Joseph,M,0,Gutierrez,1449.11628,free,"Columbia, SC",PUT,NextSong,1.540809e+12,284,Supreme Balloon,200,1541911006796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_3...",75
6,Gary Allan,Logged In,Ryann,F,0,Smith,259.83955,free,"Palestine, TX",PUT,NextSong,1.540685e+12,328,The One,200,1541930188796,Mozilla/5.0 (Macintosh; Intel Mac OS X 10.9; r...,92
7,Miracle Fortress,Logged In,Ryann,F,1,Smith,200.90730,free,"Palestine, TX",PUT,NextSong,1.540685e+12,328,Five Roses,200,1541930447796,Mozilla/5.0 (Macintosh; Intel Mac OS X 10.9; r...,92
8,Don Omar,Logged In,Ryann,F,2,Smith,261.35465,free,"Palestine, TX",PUT,NextSong,1.540685e+12,328,Cuentale,200,1541930647796,Mozilla/5.0 (Macintosh; Intel Mac OS X 10.9; r...,92
9,Jay-Z,Logged In,Ryann,F,3,Smith,212.27057,free,"Palestine, TX",PUT,NextSong,1.540685e+12,328,D'Evils,200,1541930908796,Mozilla/5.0 (Macintosh; Intel Mac OS X 10.9; r...,92


# Below code you are trying to convert to add a new column ts_ms to df .Instead just assign it to t

In [84]:
#df['ts_ms']  = pd.to_datetime(df['ts'],unit = 'ms')
#df.ts_ms.dt
t  = pd.to_datetime(df['ts'],unit = 'ms')
t.head()

0   2018-11-11 02:33:56.796
1   2018-11-11 02:36:10.796
2   2018-11-11 02:40:34.796
3   2018-11-11 04:34:01.796
4   2018-11-11 04:36:13.796
Name: ts, dtype: datetime64[ns]

## Below code is not required as you can extract hour,day,week,month,year,weekday directly from t 

In [85]:
#t =  df[df.ts_ms.notnull()]
#t=pd.DataFrame(t['ts_ms'])
#t['hour'] = t.ts_ms.dt.hour
#t['day'] = t.ts_ms.dt.day
#t['week'] = t.ts_ms.dt.week
#t['month'] = t.ts_ms.dt.month
#t['year'] = t.ts_ms.dt.year
#t['weekday'] = t.ts_ms.dt.weekday

## assign the values of 'start_time' , 'hour' , 'day' , 'week', 'month', 'year', 'weekday' in time_data. Try to to print the value for time_data aftet that.you will the all the values in the tuple.we need to convert it to dataframe by using dict funtion to zip the columns and values.

In [86]:
time_data = (df.ts,t.dt.hour,t.dt.day,t.dt.week,t.dt.month,t.dt.year,t.dt.weekday)
column_labels = ('start_time' , 'hour' , 'day' , 'week', 'month', 'year', 'weekday')

In [87]:
time_df = pd.DataFrame(dict(zip(column_labels, time_data)))
time_df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 95 entries, 0 to 94
Data columns (total 7 columns):
start_time    95 non-null int64
hour          95 non-null int64
day           95 non-null int64
week          95 non-null int64
month         95 non-null int64
year          95 non-null int64
weekday       95 non-null int64
dtypes: int64(7)
memory usage: 5.3 KB


#### Insert Records into Time Table
Implement the `time_table_insert` query in `sql_queries.py` and run the cell below to insert records for the timestamps in this log file into the `time` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `time` table in the sparkify database.

In [88]:
for i, row in time_df.iterrows():
    cur.execute(time_table_insert, list(row))
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #4: `users` Table
#### Extract Data for Users Table
- Select columns for user ID, first name, last name, gender and level and set to `user_df`

In [89]:
df = pd.read_json(path_or_buf=get_files(filepath)[2], lines = True)

In [90]:
df1 = df[['userId']].apply(pd.to_numeric) 
df2 =  df[[ 'firstName', 'lastName', 'gender','level' ]]
user_df =df2
user_df = pd.DataFrame(user_df)
user_df['user_id'] = df1

In [91]:
user_df = user_df[user_df.user_id.notnull()]
user_df

,firstName,lastName,gender,level,user_id
0,Anabelle,Simpson,F,free,69.0
1,Anabelle,Simpson,F,free,69.0
2,Anabelle,Simpson,F,free,69.0
3,Lily,Burns,F,free,32.0
4,Lily,Burns,F,free,32.0
5,Joseph,Gutierrez,M,free,75.0
6,Ryann,Smith,F,free,92.0
7,Ryann,Smith,F,free,92.0
8,Ryann,Smith,F,free,92.0
9,Ryann,Smith,F,free,92.0


#### Insert Records into Users Table
Implement the `user_table_insert` query in `sql_queries.py` and run the cell below to insert records for the users in this log file into the `users` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `users` table in the sparkify database.

In [92]:
for i, row in user_df.iterrows():
    cur.execute(user_table_insert, row)
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #5: `songplays` Table
#### Extract Data and Songplays Table
This one is a little more complicated since information from the songs table, artists table, and original log file are all needed for the `songplays` table. Since the log file does not specify an ID for either the song or the artist, you'll need to get the song ID and artist ID by querying the songs and artists tables to find matches based on song title, artist name, and song duration time.
- Implement the `song_select` query in `sql_queries.py` to find the song ID and artist ID based on the title, artist name, and duration of a song.
- Select the timestamp, user ID, level, song ID, artist ID, session ID, location, and user agent and set to `songplay_data`

#### Insert Records into Songplays Table
- Implement the `songplay_table_insert` query and run the cell below to insert records for the songplay actions in this log file into the `songplays` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songplays` table in the sparkify database.

In [93]:
df.head(2)

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,Frumpies,Logged In,Anabelle,F,0,Simpson,134.47791,free,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",PUT,NextSong,1.541044e+12,455,Fuck Kitty,200,1541903636796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",69
1,Kenny G with Peabo Bryson,Logged In,Anabelle,F,1,Simpson,264.75057,free,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",PUT,NextSong,1.541044e+12,455,By The Time This Night Is Over,200,1541903770796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",69


In [94]:
#df1=df[['ts','userId', 'level']]+ songid , artistid + df[['sessionId', 'location', 'userAgent']]
df.head()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,Frumpies,Logged In,Anabelle,F,0,Simpson,134.47791,free,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",PUT,NextSong,1.541044e+12,455,Fuck Kitty,200,1541903636796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",69
1,Kenny G with Peabo Bryson,Logged In,Anabelle,F,1,Simpson,264.75057,free,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",PUT,NextSong,1.541044e+12,455,By The Time This Night Is Over,200,1541903770796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",69
2,Biffy Clyro,Logged In,Anabelle,F,2,Simpson,189.83138,free,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",PUT,NextSong,1.541044e+12,455,God & Satan,200,1541904034796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",69
3,None,Logged In,Lily,F,0,Burns,NaN,free,"New York-Newark-Jersey City, NY-NJ-PA",GET,Home,1.540621e+12,456,None,200,1541910841796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",32
4,HIM,Logged In,Lily,F,1,Burns,212.06159,free,"New York-Newark-Jersey City, NY-NJ-PA",PUT,NextSong,1.540621e+12,456,Beautiful,200,1541910973796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",32


In [95]:
df1 = df[['userId']].apply(pd.to_numeric)
df2 =  df[['song','ts','length', 'artist','level','sessionId','location', 'userAgent' ]]
user_df =df2
user_df = pd.DataFrame(user_df)
user_df['userId'] = df1
user_df['userId']
user_df = user_df[user_df.song.notnull()]
#user_df['userId'] = user_df.userId.apply(pd.to_numeric).astype(int)
user_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 73 entries, 0 to 94
Data columns (total 9 columns):
song         73 non-null object
ts           73 non-null int64
length       73 non-null float64
artist       73 non-null object
level        73 non-null object
sessionId    73 non-null int64
location     73 non-null object
userAgent    73 non-null object
userId       73 non-null float64
dtypes: float64(2), int64(2), object(5)
memory usage: 5.7+ KB


In [96]:
df2 = (user_df.song, user_df.ts ,user_df.length, user_df.artist,user_df.level,user_df.sessionId,user_df.location,user_df.userAgent, user_df.userId)
column_names = ('song', 'ts' , 'length', 'artist','level','sessionId','location', 'userAgent', 'userId')
df3 = pd.DataFrame(dict(zip(column_names, df2)))
df3.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 73 entries, 0 to 94
Data columns (total 9 columns):
song         73 non-null object
ts           73 non-null int64
length       73 non-null float64
artist       73 non-null object
level        73 non-null object
sessionId    73 non-null int64
location     73 non-null object
userAgent    73 non-null object
userId       73 non-null float64
dtypes: float64(2), int64(2), object(5)
memory usage: 5.7+ KB


In [99]:
for index, row in df.iterrows():

    # get songid and artistid from song and artist tables
    print(row.song, row.artist, row.length)
    #print(song_select)
    cur.execute(song_select, (row.song,  row.length, row.artist))
    
    results = cur.fetchone()
    
    if results:
        songid, artistid = results
        print(songid, artistid)
    else:
        songid, artistid = None, None
    print(songplay_table_insert)
    # insert songplay record
    songplay_data = (row.ts, int(row.userId), row.level, songid, artistid, row.sessionId, row.location, row.userAgent)
    cur.execute(songplay_table_insert, songplay_data)
    conn.commit()

Fuck Kitty Frumpies 134.47791
Insert into songplays( start_time , user_id , level , song_id , artist_id , session_id , location , user_agent ) values( %s, %s, %s, %s, %s, %s, %s, %s)  ;


NameError: name 'text' is not defined

Run `test.ipynb` to see if you've successfully added records to this table.

# Close Connection to Sparkify Database

In [ ]:
conn.close()

# Implement `etl.py`
Use what you've completed in this notebook to implement `etl.py`.